In [ ]:
from flask import Flask, render_template, request, flash, redirect, url_for
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import os
import cv2
from tensorflow.keras.applications.resnet50 import preprocess_input
from flask import flash
from datetime import date
from datetime import datetime
import csv
from random import * 

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import smtplib


import json
c = ""
file_path = ""
adhar = ""
email = ""
name=""
model = load_model('bestmodel94.h5')
app = Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def home():
    return render_template('home.html')

app.config['MAX_CONTENT_LENGTH'] = 10 * 1024 * 1024

ALLOWED_EXTENSIONS = ['png', 'jpg', 'jpeg']
def allowed_file(filename):
    return '.' in filename and \
          filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
           
def init():
    global graph
    graph = tf.compat.v1.get_default_graph()
    
@app.route("/login1", methods=['GET', 'POST'])
def login1():
    return render_template('login.html')

@app.route("/homeprotect", methods=['GET', 'POST'])
def homeprotect():
    return render_template('homeprotect.html')

@app.route("/model", methods=['GET', 'POST'])
def model():
    return render_template('modeldetails.html')

@app.route("/view", methods=['GET', 'POST'])
def view():
    return render_template('viewdetails.html')

@app.route("/logout", methods=['GET', 'POST'])
def logout():
    return render_template('home.html')

@app.route("/index", methods=['GET', 'POST'])
def index():
    return render_template('index.html')

@app.route('/table')
def table():
    data1 = pd.read_csv('static/details.csv')
    data2 = pd.read_csv('static/results.csv')

    output1 = pd.merge(data1, data2, on='Adhar Number', how='inner')
    return render_template('table.html', tables=[output1.to_html(index=False)], titles=[''])

@app.route('/c',methods=['GET', 'POST'])
def c():
    data = pd.read_csv('static/results.csv') 
    result = data['Result'].value_counts()
    try:
        out=result[1]
    except:
        out=0
    return render_template('table.html',r="Total number of Covid-19 Detected Patients - "+str(out))


@app.route('/n',methods=['GET', 'POST'])
def n():
    data = pd.read_csv('static/results.csv')   
    result = data['Result'].value_counts()
    try:
        out=result[0]
    except:
        out=0
    return render_template('table.html',r="Total number of Normal Patients - "+str(out))


@app.route('/p',methods=['GET', 'POST'])
def p():
    data = pd.read_csv('static/results.csv')
    result = data['Result'].value_counts()
    try:
        out=result[2]
    except:
        out=0
    return render_template('table.html',r="Total number of Pneumonia Detected Patients - "+str(out))


@app.route("/form_login", methods=['GET', 'POST'])
def login():
    userid=request.form["id"]
    password=request.form["password"]
    f = open('static/login_detail.json','r')
    data = json.load(f)
    #data={'1234':'abcd'}
    if userid in data:
        if data[userid]==password:
            f.close()
            return render_template('index.html')
        else:
            f.close()
            return render_template('login.html', info='Invalid Password')
    else:
        f.close()
    return render_template('login.html', info='Invalid UserId')
  
def validate():
    if request.method == 'POST':
        name = request.form.get("name")
        age = request.form.get("age")
        adhar = request.form.get("adhar")
        phno = request.form.get("phno")
        #email = request.form.get("email")
        adds = request.form.get("adds")
        gen = request.form.get("gender")
        error=''
        if (len(name)==0 or len(age)==0 or len(adhar)==0 or len(phno)==0 or len(adds)==0):
            error = "One of the field is empty!"
        elif int(age)<=0 or int(age)>100:
            error ="Age Value is Incoorect"
        elif len(adhar)!=12:
            error = "Adhar number is incorrect"
        elif len(phno)!=10:
            error = "Phone number is Incorrect"
    return error,name,age,adhar,phno,adds,gen


def adddetails(name,age,adhar,phno,email,adds,gen):
    name = name
    age = int(age)
    adhar = int(adhar)
    phno = int(phno)
    email = email
    adds = adds
    gen = gen
    d = datetime.today() 
    
    row = [adhar,name,age,gen,phno,email,adds,d]

    filename = "static/details.csv"

    df = pd.read_csv(filename)
    result = df.loc[df["Adhar Number"] == adhar]
    
    filename1 = "static/results.csv"

    df1 = pd.read_csv(filename1)
    result1 = df1.loc[df1["Adhar Number"] == adhar]
    #res = result1.iloc[0,1]
    #print(res)
    
    e = ''
    if(len(result)==0):
        with open(filename, 'a') as csvfile:  
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(row)       
    else:
        if len(result1)!=0:
            d1 = result.iloc[0,7]
            d1 = datetime.strptime(d1, '%Y-%m-%d %H:%M:%S.%f')
            diff = (d-d1).days
            res = result1.iloc[0,1]
        
            if res=='Normal' or res=='Pneumonia':
                d = df.loc[df["Adhar Number"] == adhar]
                df.drop(d.index, inplace=True)
                df.to_csv(filename,index=False)
                with open(filename, 'a') as csvfile:  
                    csvwriter = csv.writer(csvfile) 
                    csvwriter.writerow(row)
            else:
                if diff>=14:
                    d = df.loc[df["Adhar Number"] == adhar]
                    df.drop(d.index, inplace=True)
                    df.to_csv(filename,index=False)
                    with open(filename, 'a') as csvfile:  
                        csvwriter = csv.writer(csvfile) 
                        csvwriter.writerow(row) 
                else:
                    e = "Upload After 14 days"
    return e

@app.route("/form_password", methods=['GET', 'POST'])
def form_password():
    return render_template('changepassword.html')

@app.route("/forgot-password", methods=['GET', 'POST'])
def changepassword():
    userid=request.form["id"]
    password1=request.form["password1"]
    password2=request.form["password2"]
    f = open('static/login_detail.json','r')
    data = json.load(f)
    if userid not in data:
        f.close()
        return render_template('changepassword.html', info='Invalid UserId')
    else:
        if password1!=password2:
            f.close()
            return render_template('changepassword.html', info='Password not matching')
        else:
            data[userid]=password1
            f = open('static/login_detail.json','w')
            json.dump(data, f)  
            f.close()
            return render_template('changepassword.html', info='Password set')

@app.route("/form_loginback", methods=['GET', 'POST'])
def form_loginback():
    return render_template('login.html')

@app.route("/details", methods=['GET', 'POST'])
def details():
    return render_template('email.html')

@app.route("/existpatient", methods=['GET', 'POST'])
def existpatient():
    return render_template('patientsignin.html')

@app.route('/signin',methods = ["POST"])
def signin():
    global email
    i=""
    email = request.form["email"] 
    filename = "static/details.csv"
    df = pd.read_csv(filename)
    result = df.loc[df["Email"] == email]
    if len(result)!=0:
        return render_template('patient_details.html')
    else:
        i = "Patient not registered yet!!"
    return render_template('patientsignin.html' , i=i) 



@app.route('/verify',methods = ["POST"])
def verify():
    global otp
    global email
    msg = MIMEMultipart()
    otp = randint(0,999999) 
    password = "uphcqbiwltuseqox"
    email = request.form["email"]   
    msg['From'] = "covid19predictionvcet@gmail.com"
    msg['To'] = email
    msg['Subject'] = " OTP Verification "  

    msg.attach(MIMEText("Your OTP: "+str(otp)))
    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.ehlo() 
    server.login('covid19predictionvcet@gmail.com', 'uphcqbiwltuseqox')
    server.sendmail(msg['From'], msg['To'] , msg.as_string())  
    server.quit()
    return render_template('verify.html')

@app.route('/validateemail',methods=["POST"])   
def validateemail():
    user_otp = request.form['otp']  
    if otp == int(user_otp):
        return render_template('patient_details.html')
    return render_template('verify.html', info="Failure, OTP does not match") 

    
@app.route('/resnet50',methods=["POST"])   
def resnet50():
    global model
    global file_path
    global adhar
    global c
    model = load_model('bestmodel92.h5')
    classes = ['Covid' ,'Normal', 'Pneumonia' ]
    try:
        image = cv2.imread(file_path)
        image = cv2.resize(image,(224,224))
        image = preprocess_input(image)
        image = image.reshape(1,224,224,3)
        pred = model.predict(image)
        preds = np.argmax(pred,axis=1)[0]
        if preds==0:
            c = 'Covid'
        elif preds==2:
            c = 'Pneumonia'
        else:
            c = 'Normal'
        dict_result={}
        for i in range(3):
            dict_result[pred[0][i]] = classes[i]
        p = "The probabilty is : "+str((max(dict_result)*100).round(2)) 
        print(p)
        res = pred[0]
        res.sort()
        res = res[::-1]
        pro = res[:3]

        prob_result = []
        class_result = []
        for i in range(3):
            prob_result.append((pro[i]*100).round(2))
            class_result.append(dict_result[pro[i]])
        predictions = {
                "class1":class_result[0],
                "class2":class_result[1],
                "class3":class_result[2],
                "prob1": prob_result[0],
                "prob2": prob_result[1],
                "prob3": prob_result[2],}
        row = [adhar,c]
        '''filename = "static/results.csv"
        with open(filename, 'a') as csvfile:  
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(row)'''
        return render_template('other_model.html', product = c, user_image = file_path, prob = predictions)
    except Exception as e:
        return "Something Went Wrong!!"
    
    
@app.route('/resnet101',methods=["POST"])   
def resnet101():
    global model
    model = load_model('bestmodelr101.h5')
    classes = ['Covid' ,'Normal', 'Pneumonia' ]
    try:
        image = cv2.imread(file_path)
        image = cv2.resize(image,(224,224))
        image = preprocess_input(image)
        image = image.reshape(1,224,224,3)
        pred = model.predict(image)
        preds = np.argmax(pred,axis=1)[0]
        if preds==0:
            c = 'Covid'
        elif preds==2:
            c = 'Pneumonia'
        else:
            c = 'Normal'
        dict_result={}
        for i in range(3):
            dict_result[pred[0][i]] = classes[i]
        p = "The probabilty is : "+str((max(dict_result)*100).round(2)) 
        print(p)
        res = pred[0]
        res.sort()
        res = res[::-1]
        pro = res[:3]

        prob_result = []
        class_result = []
        for i in range(3):
            prob_result.append((pro[i]*100).round(2))
            class_result.append(dict_result[pro[i]])
        predictions = {
                "class1":class_result[0],
                "class2":class_result[1],
                "class3":class_result[2],
                "prob1": prob_result[0],
                "prob2": prob_result[1],
                "prob3": prob_result[2],}
        row = [adhar,c]
        '''filename = "static/results.csv"
        with open(filename, 'a') as csvfile:  
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(row)'''
        return render_template('other_model.html', product = c, user_image = file_path, prob = predictions)
    except Exception as e:
        return "Something Went Wrong!!"
    
@app.route('/vgg16',methods=["POST"])   
def vgg16():
    global model
    model = load_model('bestmodelvgg16.h5')
    classes = ['Covid' ,'Normal', 'Pneumonia' ]
    try:
        image = cv2.imread(file_path)
        image = cv2.resize(image,(224,224))
        image = preprocess_input(image)
        image = image.reshape(1,224,224,3)
        pred = model.predict(image)
        preds = np.argmax(pred,axis=1)[0]
        if preds==0:
            c = 'Covid'
        elif preds==2:
            c = 'Pneumonia'
        else:
            c = 'Normal'
        dict_result={}
        for i in range(3):
            dict_result[pred[0][i]] = classes[i]
        p = "The probabilty is : "+str((max(dict_result)*100).round(2)) 
        print(p)
        res = pred[0]
        res.sort()
        res = res[::-1]
        pro = res[:3]

        prob_result = []
        class_result = []
        for i in range(3):
            prob_result.append((pro[i]*100).round(2))
            class_result.append(dict_result[pro[i]])
        predictions = {
                "class1":class_result[0],
                "class2":class_result[1],
                "class3":class_result[2],
                "prob1": prob_result[0],
                "prob2": prob_result[1],
                "prob3": prob_result[2],}
        row = [adhar,c]
        '''filename = "static/results.csv"
        with open(filename, 'a') as csvfile:  
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(row)'''
        return render_template('other_model.html', product = c, user_image = file_path, prob = predictions)
    except Exception as e:
        return "Something Went Wrong!!"
    
    
    
@app.route('/vgg19',methods=["POST"])   
def vgg19():
    global model
    model = load_model('bestmodelvgg19.h5')
    classes = ['Covid' ,'Normal', 'Pneumonia' ]
    try:
        image = cv2.imread(file_path)
        image = cv2.resize(image,(224,224))
        image = preprocess_input(image)
        image = image.reshape(1,224,224,3)
        pred = model.predict(image)
        preds = np.argmax(pred,axis=1)[0]
        if preds==0:
            c = 'Covid'
        elif preds==2:
            c = 'Pneumonia'
        else:
            c = 'Normal'
        dict_result={}
        for i in range(3):
            dict_result[pred[0][i]] = classes[i]
        p = "The probabilty is : "+str((max(dict_result)*100).round(2)) 
        print(p)
        res = pred[0]
        res.sort()
        res = res[::-1]
        pro = res[:3]

        prob_result = []
        class_result = []
        for i in range(3):
            prob_result.append((pro[i]*100).round(2))
            class_result.append(dict_result[pro[i]])
        predictions = {
                "class1":class_result[0],
                "class2":class_result[1],
                "class3":class_result[2],
                "prob1": prob_result[0],
                "prob2": prob_result[1],
                "prob3": prob_result[2],}
        row = [adhar,c]
        '''filename = "static/results.csv"
        with open(filename, 'a') as csvfile:  
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(row)'''
        return render_template('other_model.html', product = c, user_image = file_path, prob = predictions)
    except Exception as e:
        return "Something Went Wrong!!"
    
               
@app.route("/predict", methods = ['GET','POST'])
def predict():
    global adhar
    global email
    global file_path
    global c
    global name
    global model
    error,name,age,adhar,phno,adds,gen = validate()
    if error=='':
        e = adddetails(name,age,adhar,phno,email,adds,gen)
        if e == '':
            model = load_model('bestmodel92.h5')
            classes = ['Covid' ,'Normal', 'Pneumonia' ]
            if request.method == 'POST':
                file = request.files['file']
                try:
                    if file and allowed_file(file.filename):
                        f1 = os.path.join('static/images', adhar+'.jpeg')
                        f2 = os.path.join('static/images', adhar+'c.jpeg')
                        if(os.path.exists(f1)):
                            os.remove(f1) 
                            file_path = os.path.join('static/images', adhar+'c.jpeg')
                        elif (os.path.exists(f2)):
                            os.remove(f2) 
                            file_path = os.path.join('static/images', adhar+'.jpeg')
                        else:
                            file_path = os.path.join('static/images', adhar+'.jpeg')
                        file.save(file_path)
                        image = cv2.imread(file_path)
                        image = cv2.resize(image,(224,224))
                        image = preprocess_input(image)
                        image = image.reshape(1,224,224,3)
                        pred = model.predict(image)
                        preds = np.argmax(pred,axis=1)[0]
                        if preds==0:
                            c = 'Covid'
                        elif preds==2:
                            c = 'Pneumonia'
                        else:
                            c = 'Normal'
                        dict_result={}
                        for i in range(3):
                            dict_result[pred[0][i]] = classes[i]
                        p = "The probabilty is : "+str((max(dict_result)*100).round(2)) 
                        print(p)
                        res = pred[0]
                        res.sort()
                        res = res[::-1]
                        pro = res[:3]

                        prob_result = []
                        class_result = []
                        for i in range(3):
                            prob_result.append((pro[i]*100).round(2))
                            class_result.append(dict_result[pro[i]])
                        predictions = {
                              "class1":class_result[0],
                                "class2":class_result[1],
                                "class3":class_result[2],
                                "prob1": prob_result[0],
                                "prob2": prob_result[1],
                                "prob3": prob_result[2],}

                        row = [adhar,c]
                        filename = "static/results.csv"
                        df = pd.read_csv(filename)
                        result = df.loc[df["Adhar Number"] == int(adhar)]
                        if(len(result)==0):
                            with open(filename, 'a') as csvfile:  
                                csvwriter = csv.writer(csvfile) 
                                csvwriter.writerow(row)
                            return render_template('predict.html',product = c, user_image = file_path, prob = predictions)

                        else:
                            d = df.loc[df["Adhar Number"] == int(adhar)]
                            df.drop(d.index, inplace=True)
                            df.to_csv(filename,index=False)
                            with open(filename, 'a') as csvfile:  
                                csvwriter = csv.writer(csvfile) 
                                csvwriter.writerow(row)
                            return render_template('predict.html',product = c, user_image = file_path, prob = predictions)
                except Exception as e:
                    return "Something Went Wrong!!"
        else:
            return render_template('patient_details.html' , e = e) 
    else:
        return render_template('patient_details.html' , error = error)
    
@app.route("/sendmail", methods = ['GET','POST'])
def sendmail():
    global c
    global file_path
    global adahr
    global email 
    global name
    msg = MIMEMultipart()
    password = "uphcqbiwltuseqox"
    msg['From'] = "covid19predictionvcet@gmail.com"
    msg['To'] = email
    msg['Subject'] = " Results of X-ray "   
    img_data = open(file_path, 'rb').read()
    image = MIMEImage(img_data, name=os.path.basename(adhar + '.jpeg'))
    msg.attach(image)
    msg.attach(MIMEText("Dear "+name+",\n\n\n"))
    msg.attach(MIMEText("Your Test Result is "+c+"\n\n\n"))
    message=""
    if request.method == "POST":
        message = request.form.get("review")
    msg.attach(MIMEText("Thank You\n\n\n"))
    msg.attach(MIMEText(message))
    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.ehlo() 
    server.login('covid19predictionvcet@gmail.com', 'uphcqbiwltuseqox')
    server.sendmail(msg['From'], msg['To'] , msg.as_string())  
    server.quit()
    msg = MIMEMultipart()
    msg['From'] = "covid19predictionvcet@gmail.com"
    msg['To'] = "manasakmhegde135@gmail.com"
    msg['Subject'] = " Results of X-ray " 
    img_data = open(file_path, 'rb').read()
    image = MIMEImage(img_data, name=os.path.basename(adhar + '.jpeg'))
    msg.attach(image)
    msg.attach(MIMEText("Name : "+name+",\n"))
    msg.attach(MIMEText("Adhar No. : "+adhar+" \n\nResult : "+c+"\n"))
    message=""
    if request.method == "POST":
        message = request.form.get("review")
    msg.attach(MIMEText(message))
    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.ehlo() 
    server.login('covid19predictionvcet@gmail.com', 'uphcqbiwltuseqox')
    server.sendmail(msg['From'], msg['To'] , msg.as_string())  
    server.quit()
    print("Successfully Sent Email to Covid Center and %s:" % (msg['To']))
    return render_template('index.html')



if __name__ == '__main__':
    init()
    app.run(debug=False) 